# I) Python et Data Engineering

In [16]:

import pandas as pd
import json


In [17]:


# Chargement des fichiers

drugs_df = pd.read_csv('/content/drugs.csv')
clinical_trials_df = pd.read_csv('/content/clinical_trials.csv',encoding='utf-8')
pubmed_csv_df = pd.read_csv('/content/pubmed.csv')


with open('/content/pubmed.json', 'r') as f:
    pubmed_json_df = pd.DataFrame(json.load(f))


print("Drugs Data:")
print(drugs_df.head())

print("\nClinical Trials Data:")
print(clinical_trials_df.head())

print("\nPubMed CSV Data:")
print(pubmed_csv_df.head())

print("\nPubMed JSON Data:")
print(pubmed_json_df.head())


Drugs Data:
  atccode             drug
0   A04AD  DIPHENHYDRAMINE
1   S03AA     TETRACYCLINE
2   V03AB          ETHANOL
3   A03BA         ATROPINE
4   A01AD      EPINEPHRINE

Clinical Trials Data:
            id  \
0  NCT01967433   
1  NCT04189588   
2  NCT04237090   
3  NCT04237091   
4  NCT04153396   

                                                                                                                                                        scientific_title  \
0                                                                    Use of Diphenhydramine as an Adjunctive Sedative for Colonoscopy in Patients Chronically on Opioids   
1                                                                                   Phase 2 Study IV QUZYTTIR™ (Cetirizine Hydrochloride Injection) vs V Diphenhydramine   
2                                                                                                                                                                          
3  Fea

# Data cleansing

Dans cette étape de cleansing , l'idée est d'appliquer un ensemble de traitement pour nettoyer et uniformiser la donnée :

1.   Uniformisation du format des dates : les dates dans les fichiers ont plusieurs format différents ( 01/01/2019 , 2020-01-01 , "1 January 2020" ) donc il faut uniformiser le format pour pouvoir les traiter.
2. Gestion du mauvais encodage : exemple "Journal of emergency nursing\xc3\x28".
3. Suppression des espaces non nécéssaires.
4. Autres traitements possibles (non appliqués dans cet exercice): Gestion des valeurs nulles , gestion des doublons , gestion des caractères spéciaux.






In [18]:

# Fonction pour définir un format uniforme pour les dates 'YYYY-MM-DD'
def format_date(df, date_column):
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce', format=None)
    df[date_column] = df[date_column].dt.strftime('%Y-%m-%d')
    return df

clinical_trials_df = format_date(clinical_trials_df, 'date')
pubmed_csv_df = format_date(pubmed_csv_df, 'date')
pubmed_json_df = format_date(pubmed_json_df, 'date')



In [19]:

print("\nClinical Trials Data:")
print(clinical_trials_df.head())

print("\nPubMed CSV Data:")
print(pubmed_csv_df.head())

print("\nPubMed JSON Data:")
print(pubmed_json_df.head())


Clinical Trials Data:
            id  \
0  NCT01967433   
1  NCT04189588   
2  NCT04237090   
3  NCT04237091   
4  NCT04153396   

                                                                                                                                                        scientific_title  \
0                                                                    Use of Diphenhydramine as an Adjunctive Sedative for Colonoscopy in Patients Chronically on Opioids   
1                                                                                   Phase 2 Study IV QUZYTTIR™ (Cetirizine Hydrochloride Injection) vs V Diphenhydramine   
2                                                                                                                                                                          
3  Feasibility of a Randomized Controlled Clinical Trial Comparing the Use of Cetirizine to Replace Diphenhydramine in the Prevention of Reactions Related to Paclitaxel   
4       

In [20]:
# Fonction pour corriger les problèmes d'encodage
def correct_encoding(text):
    if isinstance(text, str):
        try:
            text = bytes(text, 'utf-8').decode('unicode_escape')
            text = text.encode('latin1').decode('utf-8', errors='ignore')
        except UnicodeDecodeError:
            pass
    return text

# Fonction pour appliquer le correctif sur les colonnes de texte ( str ) uniquement
def clean_text_columns(df):
    text_columns = df.select_dtypes(include=['object']).columns
    for column in text_columns:
        df[column] = df[column].apply(correct_encoding)
    return df


In [21]:


clinical_trials_df = clean_text_columns(clinical_trials_df)
pubmed_csv_df = clean_text_columns(pubmed_csv_df)
pubmed_json_df = clean_text_columns(pubmed_json_df)


pd.set_option('display.max_colwidth', None)


print(clinical_trials_df)
print(pubmed_csv_df)
print(pubmed_json_df)


            id  \
0  NCT01967433   
1  NCT04189588   
2  NCT04237090   
3  NCT04237091   
4  NCT04153396   
5  NCT03490942   
6          NaN   
7  NCT04188184   

                                                                                                                                                        scientific_title  \
0                                                                    Use of Diphenhydramine as an Adjunctive Sedative for Colonoscopy in Patients Chronically on Opioids   
1                                                                                   Phase 2 Study IV QUZYTTIR™ (Cetirizine Hydrochloride Injection) vs V Diphenhydramine   
2                                                                                                                                                                          
3  Feasibility of a Randomized Controlled Clinical Trial Comparing the Use of Cetirizine to Replace Diphenhydramine in the Prevention of Reactions Re

In [22]:
# Fonction pour supprimer les espaces non nécéssaires
def clean_text_column(column):

    return column.apply(lambda x: x.strip() if isinstance(x, str) else x)


In [23]:
# Appliquer clean_text_column sur des colonnes spécifiques dans clinical_trials_df
clinical_trials_df['scientific_title'] = clean_text_column(clinical_trials_df['scientific_title'])

# Appliquer clean_text_column sur la colonne 'title' dans pubmed_csv_df
pubmed_csv_df['title'] = clean_text_column(pubmed_csv_df['title'])

# Appliquer clean_text_column sur la colonne 'title' dans pubmed_json_df
pubmed_json_df['title'] = clean_text_column(pubmed_json_df['title'])

drugs_df['drug'] = clean_text_column(drugs_df['drug'])


In [24]:
print("\nClinical Trials Data:")
print(clinical_trials_df)

print("\nPubMed CSV Data:")
print(pubmed_csv_df)

print("\nPubMed JSON Data:")
print(pubmed_json_df)

print("\Drugs Data:")
print(drugs_df)


Clinical Trials Data:
            id  \
0  NCT01967433   
1  NCT04189588   
2  NCT04237090   
3  NCT04237091   
4  NCT04153396   
5  NCT03490942   
6          NaN   
7  NCT04188184   

                                                                                                                                                        scientific_title  \
0                                                                    Use of Diphenhydramine as an Adjunctive Sedative for Colonoscopy in Patients Chronically on Opioids   
1                                                                                   Phase 2 Study IV QUZYTTIR™ (Cetirizine Hydrochloride Injection) vs V Diphenhydramine   
2                                                                                                                                                                          
3  Feasibility of a Randomized Controlled Clinical Trial Comparing the Use of Cetirizine to Replace Diphenhydramine in the Pre

# Data pipeline


1.  Mention des médicaments


In [25]:


# Fonction pour trouver les mentions de médicaments , générer le fichier JSON en sortie, et afficher son contenu
def get_mentions_graph(df_list, titles, drugs_list,sources, output_file):
    mentions_graph = {}


    for df, title, source in zip(df_list, titles, sources):
        for drug in drugs_list:
            # Recherche des mentions dans la colonne des titres
            matches = df[df[title].str.contains(drug, case=False, na=False)]
            for _, row in matches.iterrows():
                # Créer un nœud pour chaque médicament s'il n'existe pas encore
                if drug not in mentions_graph:
                    mentions_graph[drug] = []

                # Ajouter chaque mention comme une liaison avec journal, date , titre et source
                mentions_graph[drug].append({
                    'journal': row['journal'],
                    'date': row['date'],
                    'title': row[title],
                    'type': source # la source est soit PubMed soit Clinical Trials
                })

    # Exporter directement le graphe en fichier JSON
    with open(output_file, 'w') as json_file:
        json.dump(mentions_graph, json_file, indent=4)

    # Afficher le contenu de mentions_graph
    print(json.dumps(mentions_graph, indent=4))

    print(f"Fichier JSON généré : {output_file}")



In [26]:

# Liste des médicaments
drug_names = drugs_df['drug'].tolist()

# Liste des df de données
df_list = [clinical_trials_df, pubmed_csv_df, pubmed_json_df]

# Liste des noms de colonnes de titres
titles = ['scientific_title', 'title', 'title']

sources = ['ClinicalTrials', 'PubMed', 'PubMed']

get_mentions_graph(df_list, titles, drug_names,sources, 'mentions_graph_output.json')

{
    "DIPHENHYDRAMINE": [
        {
            "journal": "Journal of emergency nursing",
            "date": "2020-01-01",
            "title": "Use of Diphenhydramine as an Adjunctive Sedative for Colonoscopy in Patients Chronically on Opioids",
            "type": "ClinicalTrials"
        },
        {
            "journal": "Journal of emergency nursing",
            "date": "2020-01-01",
            "title": "Phase 2 Study IV QUZYTTIR\u2122 (Cetirizine Hydrochloride Injection) vs V Diphenhydramine",
            "type": "ClinicalTrials"
        },
        {
            "journal": "Journal of emergency nursing",
            "date": "2020-01-01",
            "title": "Feasibility of a Randomized Controlled Clinical Trial Comparing the Use of Cetirizine to Replace Diphenhydramine in the Prevention of Reactions Related to Paclitaxel",
            "type": "ClinicalTrials"
        },
        {
            "journal": "Journal of emergency nursing",
            "date": "2019-01-01",
     

# Traitement ad-hoc

Le nom du journal qui mentionne le plus de médicaments différents:


In [27]:


# Fonction pour trouver le journal qui mentionne le plus de médicaments différents
def journal_with_most_drug_mentions(mentions_graph):
    journal_dict = {}

    # Parcourir chaque médicament et ses mentions
    for drug, mentions in mentions_graph.items():
        for mention in mentions:
            journal = mention['journal']
            # Ajouter le journal au dictionnaire
            if journal not in journal_dict:
                journal_dict[journal] = set() # Initialiser un set pour l'unicité des éléments
            journal_dict[journal].add(drug)

    # Trouver le journal avec le plus de médicaments différents
    max_drugs_journal = max(journal_dict, key=lambda j: len(journal_dict[j]))
    return max_drugs_journal, len(journal_dict[max_drugs_journal])


In [28]:

with open('mentions_graph_output.json', 'r') as json_file:
    mentions_graph = json.load(json_file)

# Appel de la fonction pour trouver le journal avec le plus de mentions
journal, count = journal_with_most_drug_mentions(mentions_graph)
print(f"Le journal qui mentionne le plus de médicaments différents est {journal} avec {count} médicaments mentionnés.")

Le journal qui mentionne le plus de médicaments différents est The journal of maternal-fetal & neonatal medicine avec 2 médicaments mentionnés.





L’ensemble des médicaments mentionnés par les mêmes
journaux référencés par les publications scientifiques (PubMed) mais non les tests cliniques (Clinical
Trials):

In [29]:
def drugs_in_same_pubmed_journals(drug, mentions_graph):
    # Trouver les journaux dans lesquels le médicament donné est mentionné (seulement PubMed)
    pubmed_journals = set()


    if drug in mentions_graph:
        for mention in mentions_graph[drug]:
            # Ne prendre en compte que les mentions dans PubMed
            if mention['type'] == 'PubMed':
                pubmed_journals.add(mention['journal']) # Le nom du journal est ajouté à pubmed_journals s'il est de type PubMed et qu'il fait partie des mentions du médicament donné

    # Trouver les autres médicaments mentionnés dans les journaux de pubmed_journals
    other_mentionned_drugs = set()

    # Parcourir les autres médicaments dans mentions_graph
    for other_drug, mentions in mentions_graph.items():
        if other_drug != drug:  # Exclure le médicament initial
            for mention in mentions_graph[other_drug]:
                # Vérifier si le médicament est mentionné dans les mêmes journaux, uniquement de PubMed
                if mention['journal'] in pubmed_journals and mention['type'] == 'PubMed':
                    other_mentionned_drugs.add(other_drug)

    return other_mentionned_drugs


In [30]:
def load_mentions_graph(file_path):
    with open(file_path, 'r') as json_file:
        return json.load(json_file)

mentions_graph = load_mentions_graph('mentions_graph_output.json')


# Exemple d'application
drug = 'TETRACYCLINE'
other_mentionned_drugs = drugs_in_same_pubmed_journals(drug, mentions_graph)

print(f"Les médicaments mentionnés dans les mêmes journaux que {drug} (uniquement PubMed) sont : {other_mentionned_drugs}")


Les médicaments mentionnés dans les mêmes journaux que TETRACYCLINE (uniquement PubMed) sont : {'ETHANOL'}


# Pour aller plus loin


Eléments à considérer pour faire évoluer le code pour de grosses volumétries de données :

1.   Partitionnement des données : si la donnés est énorme cela peut être utile de partitionner par date par exemple ou autre(s) colonne(s) pertinentes  
2.   Orchestration des pipelines avec Airflow par exemple pour organiser et automatiser l'exécution des tâches
3. Parallélisation du traitement de données
4. Indexation : Créer des index sur les colonnes utilisées pour les filtres,   les jointures afin d'accélérer les opérations de lecture et de recherche.
5. Logging , Monitoring et gestion des erreurs : utiliser des logs et des outils de monitoring pour suivre les performances des data pipelines + intégrer une bonne gestion d'erreur pour éviter de bloquer le fonctionnement du pipeline
6. Utiliser des systèmes de cache pour stocker les résultats intermédiaires d’un pipeline ou les données fréquemment consultées pour éviter de tout recalculer à chaque fois.



# II) SQL

Première partie du test:




```
SELECT date , SUM(prod_price*prod_qty) FROM TRANSACTIONS
WHERE date BETWEEN '2020-01-01' AND '2020-12-31'
GROUP BY date
ORDER BY date ASC;
```



Seconde partie du test



```
SELECT
  t.client_id,
  SUM(CASE WHEN pn.product_type = 'MEUBLE' THEN t.prod_price * t.prod_qty ELSE 0 END) AS ventes_meuble,
  SUM(CASE WHEN pn.product_type = 'DECO' THEN t.prod_price * t.prod_qty ELSE 0 END) AS ventes_deco
FROM
  TRANSACTIONS t
JOIN
  PRODUCT_NOMENCLATURE pn
ON
  t.prod_id = pn.product_id
GROUP BY
  t.client_id
;

```

